## install dependencies

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install unsloth-zoo
!pip install kagglehub
!pip install tqdm

In [ ]:
cd /gsm-dataset-orignal

[Errno 2] No such file or directory: '/gsm-dataset-orignal'
/root/.cache/kagglehub/datasets/raymanodeep


## load imports

In [ ]:
import kagglehub
import os
import json
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer
from datasets import load_dataset
from tqdm import tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## download dataset

In [ ]:
replaced_dataset_path = kagglehub.dataset_download('raymanodeep/finetune-1-prime-nos')
original_dataset_path = kagglehub.dataset_download('raymanodeep/gsm-dataset-orignal')

original_dataset_path,replaced_dataset_path

100%|██████████| 3.50M/3.50M [00:00<00:00, 185MB/s]

Extracting files...


100%|██████████| 3.30M/3.30M [00:00<00:00, 167MB/s]

Extracting files...


('/root/.cache/kagglehub/datasets/raymanodeep/gsm-dataset-orignal/versions/1',
 '/root/.cache/kagglehub/datasets/raymanodeep/finetune-1-prime-nos/versions/1')

## mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive"
os.mkdir(path = os.path.join(drive_path ,"inference_evaluation"))




Mounted at /content/drive


## set torch device

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
torch.device(device=device)


device(type='cuda')

## load replaced dataset

In [ ]:
|import json
replaced_data = []
replaced_test_data = []

with open(str(replaced_dataset_path) +"/replaced_dataset/test_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        replaced_test_data.append(json.loads(line.strip()))

with open(str(replaced_dataset_path) +"/replaced_dataset/train_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        replaced_data.append(json.loads(line.strip()))



replaced_socratic_data = []
replaced_socratic_test_data =[]
with open(str(replaced_dataset_path) +"/replaced_dataset/test_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        replaced_socratic_test_data.append(json.loads(line.strip()))


with open(str(replaced_dataset_path) +"/replaced_dataset/train_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        replaced_socratic_data.append(json.loads(line.strip()))

## load original dataset

In [ ]:
import json
orig_data = []
orig_test_data = []

with open(str(replaced_dataset_path) +"/test.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        orig_test_data.append(json.loads(line.strip()))

with open(str(replaced_dataset_path) +"/train.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        orig_data.append(json.loads(line.strip()))



orig_socratic_data = []
orig_socratic_test_data =[]
with open(str(replaced_dataset_path) +"/test_socratic.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        orig_socratic_test_data.append(json.loads(line.strip()))


with open(str(replaced_dataset_path) +"/train_socratic.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        orig_socratic_data.append(json.loads(line.strip()))

NameError: name 'replaced_dataset_path' is not defined

## llama 3

### 8B

In [ ]:
|from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token


In [ ]:
#test original
llama3_8b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_orig_test_answers.append(response)



In [ ]:

#test replaced
llama3_8b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_replaced_test_answers.append(response)


llama3_8b_test_results = {}
llama3_8b_results["test_results"]=llama3_8b_test_answers
llama3_8b_results["replaced_test_answers"] = llama3_8b_replaced_test_answers

llama3_8b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_8b")


with open( os.path.join(llama3_8b_dir,"llama3_8b_test_results.json"), "w") as outfile:
    json.dump(llama3_8b_results, outfile)



In [ ]:

#test socratic original

llama3_8b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

llama3_8b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_replaced_test_socratic_answers.append(response)


llama3_8b_test_socratic_results = {}
llama3_8b_test_socratic_results["test_results"]=llama3_8b_test_socratic_answers
llama3_8b_test_socratic_results["replaced_test_results"] = llama3_8b_replaced_test_socratic_answers

llama3_8b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_8b")


with open( os.path.join(llama3_8b_dir,"llama3_8b_test_socratic_results.json"), "w") as outfile:
    json.dump(llama3_8b_test_socratic_results, outfile)


## llama 3.1

### 8B

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token


In [ ]:
#test original
llama3_1_8b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1_8b_orig_test_answers.append(response)



In [ ]:

#test replaced
llama3_1_8b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1_8b_replaced_test_answers.append(response)


llama3_1_8b_test_results = {}
llama3_1_8b_test_results["test_results"]=llama3_1_8b_test_answers
llama3_1_8b_test_results["replaced_test_answers"] = llama3_1_8b_replaced_test_answers

llama3_1_8b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_1_8b")


with open( os.path.join(llama3_1_8b_dir,"llama3_1_8b_test_results.json"), "w") as outfile:
    json.dump(llama3_1_8b_test_results, outfile)



In [ ]:

#test socratic original

llama3_1_8b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1_8b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

llama3_1_8b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1_8b_replaced_test_socratic_answers.append(response)


llama3_1_8b_test_socratic_results = {}
llama3_1_8b_test_socratic_results["test_results"]=llama3_1_8b_test_socratic_answers
llama3_1_8b_test_socratic_results["finetuned_results"] = llama3_1_8b_replaced_test_socratic_answers

llama3_1_8b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_1_8b")


with open( os.path.join(llama3_1_8b_dir,"llama3_1_8b_test_socratic_results.json"), "w") as outfile:
    json.dump(llama3_1_8b_test_socratic_results, outfile)


## llama 3.2

### 1B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
llama3_2_1b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_1b_orig_test_answers.append(response)



In [ ]:

#test replaced
llama3_2_1b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_1b_replaced_test_answers.append(response)


llama3_2_1b_test_results = {}
llama3_2_1b_test_results["test_results"]=llama3_2_1b_test_answers
llama3_2_1b_test_results["replaced_test_answers"] = llama3_2_1b_replaced_test_answers

llama3_2_1b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_2_1b")


with open( os.path.join(llama3_2_1b_dir,"llama3_2_1b_test_results.json"), "w") as outfile:
    json.dump(llama3_2_1b_test_results, outfile)



In [ ]:

#test socratic original

llama3_2_1b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_1b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

llama3_2_1b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_1b_replaced_test_socratic_answers.append(response)


llama3_2_1b_test_socratic_results = {}
llama3_2_1b_test_socratic_results["test_results"]=llama3_2_1b_test_socratic_answers
llama3_2_1b_test_socratic_results["replaced_test_results"] = llama3_2_1b_replaced_test_socratic_answers

llama3_2_1b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_2_1b")


with open( os.path.join(llama3_2_1b_dir,"llama3_2_1b_test_socratic_results.json"), "w") as outfile:
    json.dump(llama3_2_1b_test_socratic_results, outfile)


### 3B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
llama3_2_3b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_3b_orig_test_answers.append(response)



In [ ]:

#test replaced
llama3_2_3b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_3b_replaced_test_answers.append(response)


llama3_2_3b_test_results = {}
llama3_2_3b_test_results["test_results"]=llama3_2_3b_test_answers
llama3_2_3b_test_results["replaced_test_answers"] = llama3_2_3b_replaced_test_answers

llama3_2_3b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_2_3b")


with open( os.path.join(llama3_2_3b_dir,"llama3_2_3b_test_results.json"), "w") as outfile:
    json.dump(llama3_2_3b_test_results, outfile)



In [ ]:

#test socratic original

llama3_2_3b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_3b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

llama3_2_3b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_2_3b_replaced_test_socratic_answers.append(response)


llama3_2_1b_test_socratic_results = {}
llama3_2_1b_test_socratic_results["test_results"]=llama3_2_3b_test_socratic_answers
llama3_2_1b_test_socratic_results["replaced_test_results"] = llama3_2_3b_replaced_test_socratic_answers

llama3_2_3b_dir = os.path.join(drive_path ,"inference_evaluation" , "llama3_2_3b")


with open( os.path.join(llama3_2_3b_dir,"llama3_2_1b_test_socratic_results.json"), "w") as outfile:
    json.dump(llama3_2_1b_test_socratic_results, outfile)


## minstral

### 7B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
minstral_7b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_7b_orig_test_answers.append(response)



In [ ]:

#test replaced
minstral_7b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_7b_replaced_test_answers.append(response)


minstral_7b_test_results = {}
minstral_7b_test_results["test_results"]=minstral_7b_orig_test_answers
minstral_7b_test_results["replaced_test_answers"] = minstral_7b_replaced_test_answers

minstral_7b_dir = os.path.join(drive_path ,"inference_evaluation" , "minstral_7b")


with open( os.path.join(minstral_7b_dir,"minstral_7b_test_results.json"), "w") as outfile:
    json.dump(minstral_7b_test_results, outfile)



In [ ]:

#test socratic original


minstral_7b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_7b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

minstral_7b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_7b_replaced_test_socratic_answers.append(response)


minstral_7b_test_socratic_results = {}
minstral_7b_test_socratic_results["test_results"] = minstral_7b_test_socratic_answers
minstral_7b_test_socratic_results["replaced_test_results"] = minstral_7b_replaced_test_socratic_answers

minstral_7b_dir = os.path.join(drive_path ,"inference_evaluation" , "minstral_7b")


with open( os.path.join(minstral_7b_dir,"minstral_7b_test_socratic_results.json"), "w") as outfile:
    json.dump(minstral_7b_test_socratic_results, outfile)


### 12B (nemo 2407)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
minstral_12b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_12b_orig_test_answers.append(response)



In [ ]:

#test replaced
minstral_12b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_12b_replaced_test_answers.append(response)


minstral_12b_test_results = {}
minstral_12b_test_results["test_results"]=minstral_12b_orig_test_answers
minstral_12b_test_results["replaced_test_answers"] = minstral_12b_replaced_test_answers

minstral_12b_dir = os.path.join(drive_path ,"inference_evaluation" , "minstral_12b")


with open( os.path.join(minstral_12b_dir,"minstral_12b_test_results.json"), "w") as outfile:
    json.dump(minstral_12b_test_results, outfile)



In [ ]:

#test socratic original


minstral_12b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_12b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

minstral_12b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  minstral_12b_replaced_test_socratic_answers.append(response)


minstral_12b_test_socratic_results = {}
minstral_12b_test_socratic_results["test_results"] = minstral_12b_test_socratic_answers
minstral_12b_test_socratic_results["replaced_test_results"] = minstral_12b_replaced_test_socratic_answers

minstral_12b_dir = os.path.join(drive_path ,"inference_evaluation" , "minstral_12b")


with open( os.path.join(minstral_12b_dir,"minstral_12b_test_socratic_results.json"), "w") as outfile:
    json.dump(minstral_12b_test_socratic_results, outfile)


## qwen2

### 1.5B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "unsloth/Qwen2-1.5b-bnb-4bit", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
qwen_2_1d5b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_1d5b_orig_test_answers.append(response)



In [ ]:

#test replaced
qwen_2_1d5b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_1d5b_replaced_test_answers.append(response)


qwen_2_1d5b_test_results = {}
qwen_2_1d5b_test_results["test_results"]=qwen_2_1d5b_orig_test_answers
qwen_2_1d5b_test_results["replaced_test_answers"] = qwen_2_1d5b_replaced_test_answers

qwen_2_1d5b_dir = os.path.join(drive_path ,"inference_evaluation" , "qwen_2_1d5b")


with open( os.path.join(minstral_7b_dir,"qwen_2_1d5b_test_results.json"), "w") as outfile:
    json.dump(qwen_2_1d5b_test_results, outfile)



In [ ]:

#test socratic original


qwen_2_1d5b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_1d5b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

qwen_2_1d5b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_1d5b_replaced_test_socratic_answers.append(response)


qwen_2_1d5b_test_socratic_results = {}
qwen_2_1d5b_test_socratic_results["test_results"] = qwen_2_1d5b_test_socratic_answers
qwen_2_1d5b_test_socratic_results["replaced_test_results"] = qwen_2_1d5b_replaced_test_socratic_answers

qwen_2_1d5b_dir = os.path.join(drive_path ,"inference_evaluation" , "qwen_2_1d5b")


with open( os.path.join(qwen_2_1d5b_dir,"qwen_2_1d5b_test_socratic_results.json"), "w") as outfile:
    json.dump(qwen_2_1d5b_test_socratic_results, outfile)


### 7B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-7B", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
qwen_2_7b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_7b_orig_test_answers.append(response)



In [ ]:

#test replaced
qwen_2_7b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_7b_replaced_test_answers.append(response)


qwen_2_7b_test_results = {}
qwen_2_7b_test_results["test_results"]=qwen_2_7b_orig_test_answers
qwen_2_7b_test_results["replaced_test_answers"] = qwen_2_7b_replaced_test_answers

qwen_2_7b_dir = os.path.join(drive_path ,"inference_evaluation" , "qwen_2_7b")


with open( os.path.join(qwen_2_7b_dir,"qwen_2_7b_test_results.json"), "w") as outfile:
    json.dump(qwen_2_7b_test_results, outfile)



In [ ]:

#test socratic original


qwen_2_7b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_7b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

qwen_2_7b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  qwen_2_7b_replaced_test_socratic_answers.append(response)


qwen_2_7b_test_socratic_results = {}
qwen_2_7b_test_socratic_results["test_results"] = qwen_2_7b_test_socratic_answers
qwen_2_7b_test_socratic_results["replaced_test_results"] = qwen_2_7b_replaced_test_socratic_answers

qwen_2_7b_dir = os.path.join(drive_path ,"inference_evaluation" , "qwen_2_7b")


with open( os.path.join(qwen_2_7b_dir,"qwen_2_7b_test_socratic_results.json"), "w") as outfile:
    json.dump(qwen_2_7b_test_socratic_results, outfile)


## GEMMA 2

### 2B

In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


In [ ]:
#test original
gemma_2_2b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_2b_orig_test_answers.append(response)



In [ ]:

#test replaced
gemma_2_2b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_2b_replaced_test_answers.append(response)


gemma_2_2b_test_results = {}
gemma_2_2b_test_results["test_results"]=gemma_2_2b_orig_test_answers
gemma_2_2b_test_results["replaced_test_answers"] = gemma_2_2b_replaced_test_answers

gemma_2_2b_dir = os.path.join(drive_path ,"inference_evaluation" , "gemma_2_2b")


with open( os.path.join(gemma_2_2b_dir,"gemma_2_2b_test_results.json"), "w") as outfile:
    json.dump(gemma_2_2b_test_results, outfile)



In [ ]:

#test socratic original


gemma_2_2b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_2b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

gemma_2_2b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_2b_replaced_test_socratic_answers.append(response)


gemma_2_2b_test_socratic_results = {}
gemma_2_2b_test_socratic_results["test_results"] = gemma_2_2b_test_socratic_answers
gemma_2_2b_test_socratic_results["replaced_test_results"] = gemma_2_2b_replaced_test_socratic_answers

gemma_2_2b_dir = os.path.join(drive_path ,"inference_evaluation" , "gemma_2_2b")


with open( os.path.join(gemma_2_2b_dir,"gemma_2_2b_test_socratic_results.json"), "w") as outfile:
    json.dump(gemma_2_2b_test_socratic_results, outfile)


### 9B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#test original
gemma_2_9b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_9b_orig_test_answers.append(response)



In [ ]:

#test replaced
gemma_2_9b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_9b_replaced_test_answers.append(response)


gemma_2_9b_test_results = {}
gemma_2_9b_test_results["test_results"]=gemma_2_9b_orig_test_answers
gemma_2_9b_test_results["replaced_test_answers"] = gemma_2_9b_replaced_test_answers

gemma_2_9b_dir = os.path.join(drive_path ,"inference_evaluation" , "gemma_2_9b")


with open( os.path.join(gemma_2_9b_dir,"gemma_2_9b_test_results.json"), "w") as outfile:
    json.dump(gemma_2_9b_test_results, outfile)



In [ ]:

#test socratic original


gemma_2_9b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_9b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

gemma_2_9b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  gemma_2_9b_replaced_test_socratic_answers.append(response)


gemma_2_9b_test_socratic_results = {}
gemma_2_9b_test_socratic_results["test_results"] = gemma_2_9b_test_socratic_answers
gemma_2_9b_test_socratic_results["replaced_test_results"] = gemma_2_9b_replaced_test_socratic_answers

gemma_2_9b_dir = os.path.join(drive_path ,"inference_evaluation" , "gemma_2_9b")


with open( os.path.join(gemma_2_9b_dir,"gemma_2_9b_test_socratic_results.json"), "w") as outfile:
    json.dump(gemma_2_9b_test_socratic_results, outfile)


## SmolLM 2

### 1.7B

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-1.7B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



In [ ]:
#test original
smollm_1n7b_orig_test_answers = []

for i in tqdm(range(len(orig_test_data))):

  item = orig_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  smollm_1n7b_orig_test_answers.append(response)



In [ ]:

#test replaced
smollm_1n7b_replaced_test_answers = []

for i in tqdm(range(len(replaced_test_data))):

  item = replaced_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  smollm_1n7b_replaced_test_answers.append(response)


smollm_1n7b_test_results = {}
smollm_1n7b_test_results["test_results"]=smollm_1n7b_orig_test_answers
smollm_1n7b_test_results["replaced_test_answers"] = smollm_1n7b_replaced_test_answers

smollm_1n7b_dir = os.path.join(drive_path ,"inference_evaluation" , "smollm_1n7b")


with open( os.path.join(smollm_1n7b_dir,"smollm_1n7b_test_results.json"), "w") as outfile:
    json.dump(smollm_1n7b_test_results, outfile)



In [ ]:

#test socratic original


smollm_1n7b_test_socratic_answers = []
for i in tqdm(range(len(orig_socratic_test_data))):

  item = orig_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  smollm_1n7b_test_socratic_answers.append(response)


In [ ]:
#test socratic replaced original

smollm_1n7b_replaced_test_socratic_answers = []
for i in tqdm(range(len(replaced_socratic_test_data))):

  item = replaced_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  smollm_1n7b_replaced_test_socratic_answers.append(response)


smollm_1n7b_test_socratic_results = {}
smollm_1n7b_test_socratic_results["test_results"] = smollm_1n7b_test_socratic_answers
smollm_1n7b_test_socratic_results["replaced_test_results"] = smollm_1n7b_replaced_test_socratic_answers

smollm_1n7b_dir = os.path.join(drive_path ,"inference_evaluation" , "smollm_1n7b")


with open( os.path.join(smollm_1n7b_dir,"smollm_1n7b_test_socratic_results.json"), "w") as outfile:
    json.dump(smollm_1n7b_test_socratic_results, outfile)
